# ReAct

Ein _ReAct Agent_ ist ein KI-System, das "Reasoning and Acting" (Denken und Handeln) kombiniert. Dieses Verfahren basiert auf einem Paper: https://react-lm.github.io/.

- Abwechslung von Denken und Handeln: Der Agent wechselt zwischen Denkphasen, in denen er plant und schlussfolgert, und Handlungsphasen, in denen er Aktionen ausführt.
- Schrittweise Problemlösung: Komplexe Aufgaben werden in kleinere Schritte zerlegt und sequentiell bearbeitet.
- Anpassungsfähigkeit: Der Agent kann seinen Ansatz basierend auf Zwischenergebnissen oder neuen Informationen anpassen.
- Werkzeuge: ReAct Agenten können oft externe Tools oder Datenquellen in ihren Problemlösungsprozess einbinden.

Der Vorteil von diesem Verfahren ist, dass die Ergebnisse meist sehr viel besser sind, als wenn man einfach nur in einem One-Shot versucht die finale Antwort zu bekommen.

Jetzt werden wir noch tiefer in die automatische Trickkiste von LangChain graben, denn schlaue Leute haben sich schon Gedanken gemacht und wir machen uns das zunutze!
Wie zuvor bekommt der Agent Tools an die Hand. Wir nutzen einfach die Tools, die wir im letzten Abschnitt verfügbar gemacht haben.

Starten wir also mit der Initialisierung.

In [ ]:
!pip install langchain langchain_openai python-dotenv duckduckgo-search wikipedia langchainhub

In [ ]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

search = DuckDuckGoSearchResults()

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

Jetzt werden wir den Agenten anlegen (lassen). Dabei nutzen wir alles an Automatismus, der geboten wird. So holen wir sogar den Prompt von einem _Hub_. Auf diesem Hub (https://smith.langchain.com/hub) gibt es eine Menge Usergenerierter-Prompts, die man bequem mit einem Befehl holen kann.

Zusätzlich nutzen wir eine Hilfsfunktion, um einen ReAct-Agenten zu erstellen.

In [ ]:
# Wir holen uns den Prompt aus eine online Repository: https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Jetzt wo wir den Executor haben, können wir auch den Agenten schon mal mit der ersten Aufgabe beschäftigen.

In [ ]:
agent_executor.invoke({"input": "Was ist LangChain?"})

Das war eine relativ einfache Aufgabe. Aber man sieht schon, dass anstelle, dass der Agent einfach eine Suche lostritt, sich erstmal "Gedanken" macht, wie man zu der Lösung kommen könnte.

Lass uns den Agenten mit etwas komplizierteren Aufgabe beschäftigen.

In [11]:
agent_executor.invoke({"input": "Ich habe einige Holz-Paletten rumstehen und möchte daraus eine Gartenhütte bauen. Wie geht das?"})



> Entering new AgentExecutor chain...
Ich sollte nach Informationen suchen, wie man aus Holzpaletten eine Gartenhütte bauen kann.

Action: duckduckgo_results_json
Action Input: "Holzpaletten Gartenhütte bauen Anleitung"[snippet: Schritt-für-Schritt-Anleitung Gartenhaus aus Paletten bauen. Wenn Sie ein Häuschen für den Garten bauen wollen und dabei auf Ziegelsteine, Bagger und anderes verzichten möchten, dann bietet sich ein Gartenhäuschen aus Europaletten an. Das ist nicht nur einfach, sondern auch voll trendy., title: Gartenhaus aus Paletten - so bauen Sie es selbst - HELPSTER, link: https://www.helpster.de/gartenhaus-aus-paletten-so-bauen-sie-es-selbst_232200], [snippet: Aus Europaletten kannst du viele unterschiedliche Gartenmöbel bauen. Besonders unkompliziert sind niedrige Sitzmöbel, eine Sitzecke oder kleine Tische für deine Terrasse sowie Hochbeete, Outdoor-Theken und Schaukeln für den Garten. Der Bau der Gartenmöbel ist mit einer detaillierten DIY-Anleitung auch für Anfänger 

{'input': 'Ich habe einige Holz-Paletten rumstehen und möchte daraus eine Gartenhütte bauen. Wie geht das?',
 'output': 'Du kannst eine Schritt-für-Schritt-Anleitung zum Bau einer Gartenhütte aus Holzpaletten auf der folgenden Seite finden: [Helpster - Gartenhaus aus Paletten bauen](https://www.helpster.de/gartenhaus-aus-paletten-so-bauen-sie-es-selbst_232200).'}

In meinem Fall hat der Agent ganz schnell eine Anleitung gefunden und mir als Ergebnis präsentiert. Wo der Agent glänzt, ist wenn man eine Aufgabe vergibt, die nicht einfach durch eine einfache Suche direkt erledigt werden kann. Häufiger Usecase ist Recherchearbeit.

